# Performance Benchmarking vs DeconvolutionLab2

The purpose of this notebook is to examine performance differences between Flowdec and DeconvolutionLab2 on arbitrary test data.

**Note**: This example is only for illustration and will not be reproducible as-is unless you build this [DeconvolutionLab2 Fork](https://github.com/hadim/DeconvolutionLab2) with CUDA support locally and change the DL2_PATH variable below.

At TOW, this was executed on a system with these specs:
- Windows 10 Pro
- 64G RAM
- Intel Xeon CPU (x2)
- Nvidia GTX 1080 GPU (x2)

In [69]:
import tempfile, os, warnings, timeit
import pandas as pd
import numpy as np
from skimage import io
from flowdec import data as fd_data
from flowdec import psf as fd_psf
from flowdec import restoration as fd_restoration

DL2_PATH = os.path.join(os.path.expanduser('~'), 
    'repos', 'misc', 'DeconvolutionLab2', 'target', 'DeconcolutionLab2_-0.1.0-SNAPSHOT.jar')
N_ITER = 25 # Number of RL iterations

In [38]:
# Generate a PSF and some random data to test executions times with
kernel = fd_psf.GibsonLanni(size_x=1024, size_y=1024, size_z=11).generate().astype(np.float32)
np.random.seed(1)
data = np.random.poisson(lam=100, size=kernel.shape).astype(kernel.dtype)
data.shape, kernel.shape, data.dtype, kernel.dtype

((11, 1024, 1024), (11, 1024, 1024))

In [40]:
# Generate temporary files to store image data within
data_file, kernel_file = tempfile.mktemp('.tif', 'data-'), tempfile.mktemp('.tif', 'kernel-')

# Ignore low-contrast image warnings from skimage
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    io.imsave(data_file, data)
    io.imsave(kernel_file, kernel)
    
os.path.basename(data_file), os.path.basename(kernel_file)

('data-beuslrkm.tif', 'kernel-cifeljyp.tif')

## Benchmarking Functions

In [41]:
# Deconvolution Lab 2 Functions

# Setup system call to execute DL2 CLI
dl2_cmd = "java -cp {jar} DeconvolutionLab2 Run -image file {data} -psf file {psf} -algorithm RL {niter}"\
    .format(jar=DL2_PATH, data=data_file, psf=kernel_file, niter=N_ITER)
     
def run_dl2():
    !$dl2_cmd

In [64]:
# Flowdec Functions

algo = fd_restoration.RichardsonLucyDeconvolver(data.ndim).initialize()
acq = fd_data.Acquisition(data=data, kernel=kernel)

def run_flowdec_best_case():
    """ Use pre-defined TF graph and memory-resident data """
    algo.run(acq, niter=N_ITER)
    
def run_flowdec_worst_case():
    """ Redefine TF graph and load data from files """
    fd_restoration.RichardsonLucyDeconvolver(data.ndim).initialize().run(
        fd_data.Acquisition(data=io.imread(data_file), kernel=io.imread(kernel_file)),
        niter=N_ITER
    )

In [88]:
def benchmark(fn, n):
    times = np.array(timeit.repeat(fn, repeat=n, number=1)) * 1000
    return 'mean ± std  : {:.2f} ms ± {:.2f} ms\n'\
            'min - max   : {:.2f} ms - {:.2f} ms\n'\
            'n iterations: {}'\
        .format(times.mean(), times.std(), times.min(), times.max(), n)

## Benchmark Results

In [89]:
print(benchmark(run_flowdec_best_case, 10))

mean ± std  : 676.34 ms ± 10.17 ms
min - max   : 656.36 ms - 687.61 ms
n iterations: 10


In [91]:
print(benchmark(run_flowdec_worst_case, 10))

mean ± std  : 757.22 ms ± 19.16 ms
min - max   : 740.93 ms - 802.08 ms
n iterations: 10


In [94]:
summary = benchmark(run_dl2, 10)

Starting run
Log   	 1692.04 us 	   30.7Mb  	 AcademicFFT Added
JTransformsLibrary
Log   	 30.05 ms 	   36.1Mb  	 JTransforms Added
Log   	 32.20 ms 	   36.1Mb  	 FFTW Widsom not found in : C:\Users\User\repos\misc\DeconvolutionLab2\target\FFTW\
Log   	 32.54 ms 	   36.1Mb  	 FFTW Widsom not found in : C:\Users\User\repos\misc\DeconvolutionLab2\target\
Log   	 32.81 ms 	   36.1Mb  	 FFTW Widsom not found in : C:\Users\User\FFTW\
Log   	 33.06 ms 	   36.1Mb  	 FFTW Widsom not found in : C:\Users\User\
Log   	 33.27 ms 	   36.1Mb  	 FFTW not found
JCuFFTLibrary
Log   	 948.92 ms 	   36.1Mb  	 JCuFFT Added
Log   	 577.38 ms 	  154.9Mb  	 Load C:\Users\User\AppData\Local\Temp\data-beuslrkm.tif
Log   	 578.32 ms 	  154.9Mb  	 Open image file C:\Users\User\AppData\Local\Temp\data-beuslrkm.tif
Log   	 578.73 ms 	  154.9Mb  	 Image: 1024x1024x11 
Log   	 700.85 ms 	  244.4Mb  	 Load C:\Users\User\AppData\Local\Temp\kernel-cifeljyp.tif
Log   	 702.02 ms 	  244.4Mb  	 Open PSF file C:\Users\User

In [95]:
print(summary)

mean ± std  : 40263.56 ms ± 820.71 ms
min - max   : 39318.80 ms - 41679.61 ms
n iterations: 10
